In [3]:
%pip install -q langchain nemoguardrails langchain-openai

  Obtaining dependency information for nemoguardrails from https://files.pythonhosted.org/packages/78/06/2f4a3c5e2c82ebd95f229ec6cf1f13655846cdadb37f6878033734fbbc61/nemoguardrails-0.4.0-py3-none-any.whl.metadata
  Using cached aiohttp-3.8.4-cp311-cp311-macosx_11_0_arm64.whl (332 kB)
  Obtaining dependency information for langchain from https://files.pythonhosted.org/packages/80/59/9fe4cffec5617fc5d52e79889173947b2451bf660dc96ef54e2083cdafc4/langchain-0.0.251-py3-none-any.whl.metadata
  Using cached typer-0.7.0-py3-none-any.whl (38 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 2.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.5/647.5 kB 3.1 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Obtaining dependency information for fastapi==0.96.0 from https://files.pythonhosted.org/packages/b4/61/b62cd764a6d23220e5e8195405f6926643a41b088d3de550407e8a4edd80/fastapi-0.96.0-py3-none-any.whl.metadata
  Using cached uvicorn-0.22.0-p

In [4]:
# https://docs.getguardrails.ai/integrations/langchain/
# https://github.com/NVIDIA/NeMo-Guardrails/blob/main/examples/demo_chain_with_guardrails.py

# SPDX-FileCopyrightText: Copyright (c) 2023 NVIDIA CORPORATION & AFFILIATES. All rights reserved.
# SPDX-License-Identifier: Apache-2.0
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

"""Example of using a QnA chain with guardrails."""
import logging
import os

from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma

from nemoguardrails import LLMRails, RailsConfig

logging.basicConfig(level=logging.INFO)

COLANG_CONFIG = """
define user express greeting
  "hi"

define user express insult
  "You are stupid"

# Basic guardrail against insults.
define flow
  user express insult
  bot express calmly willingness to help

# Here we use the QA chain for anything else.
define flow
  user ...
  $answer = execute qa_chain(query=$last_user_message)
  bot $answer

"""

YAML_CONFIG = """
models:
  - type: main
    engine: openai
    model: text-davinci-003
"""


def _get_qa_chain(llm):
    """Initializes a QA chain using the jobs report.

    It uses OpenAI embeddings.
    """
    loader = TextLoader(
        os.path.join(
            os.path.dirname(__file__),
            "..",
            "examples/grounding_rail/kb/report.md",
        )
    )
    docs = loader.load()
    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
    texts = text_splitter.split_documents(docs)

    embeddings = OpenAIEmbeddings()
    docsearch = Chroma.from_documents(texts, embeddings)

    qa_chain = RetrievalQA.from_chain_type(
        llm=llm, chain_type="stuff", retriever=docsearch.as_retriever()
    )

    return qa_chain


def demo():
    """Demo of using a chain as a custom action."""
    config = RailsConfig.from_content(COLANG_CONFIG, YAML_CONFIG)
    app = LLMRails(config)

    # Create and register the chain directly as an action.
    qa_chain = _get_qa_chain(app.llm)
    app.register_action(qa_chain, name="qa_chain")

    # Change to mode here to experiment with the multiple ways of using the chain

    # mode = "chain"
    mode = "chain_with_guardrails"
    # mode = "chat_with_guardrails"

    if mode == "chain":
        query = "What is the current unemployment rate?"
        result = qa_chain.run(query)

        print(result)

    elif mode == "chain_with_guardrails":
        history = [
            {"role": "user", "content": "What is the current unemployment rate?"}
        ]
        result = app.generate(messages=history)
        print(result)

    elif mode == "chat_with_guardrails":
        history = []
        while True:
            user_message = input("> ")

            history.append({"role": "user", "content": user_message})
            bot_message = app.generate(messages=history)
            history.append(bot_message)

            # We print bot messages in green.
            print(f"\033[92m{bot_message['content']}\033[0m")


if __name__ == "__main__":
    demo()

/Users/patrick.debois/Library/Caches/pypoetry/virtualenvs/london-devops-VW7lFx7f-py3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO:nemoguardrails.actions.action_dispatcher:Initializing action dispatcher
INFO:nemoguardrails.actions.action_dispatcher:Adding retrieve_relevant_chunks to actions
INFO:nemoguardrails.actions.action_dispatcher:Adding check_jailbreak to actions
INFO:nemoguardrails.actions.action_dispatcher:Adding output_moderation_v2 to actions
INFO:nemoguardrails.actions.action_dispatcher:Adding output_moderation to actions
INFO:nemoguardrails.actions.action_dispatcher:Adding wolfram_alpha_request to actions
INFO:nemoguardrails.actions.action_dispatcher:Added summarize_document to actions
INFO:nemoguardrails.actions.action_dispatcher:Added apify to actions
INFO:nemoguardrails.actions.

NameError: name '__file__' is not defined